# &#x1F4DD; REPORT

[CompMeth](https://www.eurecom.fr/en/course/compmeth-2024spring) Spring 2024

Computational methods in digital communications provide a selection of hands-on experiments in programming and implementation techniques for high-performance computing applied to telecommunications. Students will learn architecture concepts and how to optimize software to efficiently implement different types of algorithms on software-based systems.

Pedagogical Outcomes: Students will gain knowledge of some generic digital programming methods and a deeper understanding of a topic in the context of a mini-project. They will also learn to work in teams and to synthesize their understanding through a presentation at the end of the course in front of all students.

---

#### **&#x1F516;** **Project** ___The Open Air Interface___

Project Location: [openairinterface5g](https://gitlab.eurecom.fr/oai/openairinterface5g)

Sub Project: [DMRS Channel Estimation Parallelization](https://gitlab.eurecom.fr/oai/openairinterface5g/-/tree/dmrs_channel_estimation_parallelization/openair1/PHY/NR_ESTIMATION)

Description: Using the current OAI's custom [Thread Pool](https://gitlab.eurecom.fr/oai/openairinterface5g/-/blob/develop/common/utils/threadPool/thread-pool.md), parralelize the existing NR (New Radio) UL (UpLink) Channel Estimation.

---

In the context of an uplink channel in a wireless communication system such as LTE (Long Term Evolution) or 5G NR (New Radio), DMRS (Demodulation Reference Signal) channel estimation is used to estimate the characteristics of the uplink channel. DMRS channel estimation in the uplink is a vital process in wireless communication systems, enabling the base station to accurately estimate the uplink channel conditions. This estimation allows for coherent demodulation, adaptive resource allocation, and advanced MIMO techniques, ensuring efficient and reliable communication.



#### **&#x1F516;** **(&#x61;)** ___Coding Practice___

---

Caution has been used not to break the existing `functionning` code. The coding effort relies on 3 steps:

### **(&#x31;)** Separating the main channel estimation code per antenna

- `function nr_pusch_channel_estimation()` [&#x1F4DC; Source Code](https://gitlab.eurecom.fr/oai/openairinterface5g/-/blob/dmrs_channel_estimation_parallelization/openair1/PHY/NR_ESTIMATION/nr_ul_channel_estimation.c?ref_type=heads#L521)

```c
int nr_pusch_channel_estimation(PHY_VARS_gNB *gNB, ...)
{

    //------------------generate DMRS------------------//
    <some code here>
    
    for (int aarx=0; aarx<gNB->frame_parms.nb_antennas_rx; aarx++) {
    
        // refactor the channel estimation code to a function
        
    }

 return 0;
}

```

- new function `inner_channel_estimation` {[&#x1F4DC; Source Code](https://gitlab.eurecom.fr/oai/openairinterface5g/-/blob/dmrs_channel_estimation_parallelization/openair1/PHY/NR_ESTIMATION/nr_ul_channel_estimation.c?ref_type=heads#L70)

```c
static void inner_channel_estimation(void *arg) {
    ...
}
```

&#x1F4A1; The code is still sequential

### **(&#x32;)** Implement the Thread Pool using the new `inner_channel_estimation` function


- Initialize `local` data to be passed to the threadpool,

```c
 int nb_antennas_rx = gNB->frame_parms.nb_antennas_rx;
 delay_t *delays[nb_antennas_rx];
 memset(delays, 0, sizeof(*delays));

 uint64_t noises_amp2[nb_antennas_rx];
 memset(noises_amp2, 0, sizeof(noises_amp2));
```

- In the antenna loop,
    - call the TPool, by passing the function to the worker
    - pass data `(*dtata)` to the worker
    - wait for the TPool join to complete

```c
for (int aarx=0; aarx<gNB->frame_parms.nb_antennas_rx; aarx++) {

   notifiedFIFO_elt_t *req = newNotifiedFIFO_elt(sizeof(puschAntennaProc_t), aarx, &gNB->respPuschAarx, &inner_channel_estimation); // create a job for Tpool
   puschAntennaProc_t *rdata = (puschAntennaProc_t*)NotifiedFifoData(req); // data for the job

   // Local init in the current loop
   rdata->aarx = aarx;
   rdata->noises_amp2 = noises_amp2;   // Placeholder for noise amplitude squared update
   rdata->delays = delays;

   ...

   // Call the inner_channel_estimation function
   //   inner_channel_estimation(rdata);
   pushTpool(&gNB->threadPool, req);
   gNB->nbAarx++;


   // value update of rdata to be passed to the next inner call
   delay->est_delay = rdata->delay->est_delay;  // Placeholder for estimated delay update
   ...

} // Antenna Loop

 // ThreadPool Join (also called Rendez-Vous)
 while (gNB->nbAarx > 0) {
   notifiedFIFO_elt_t *req = pullTpool(&gNB->respPuschAarx, &gNB->threadPool);
   gNB->nbAarx--;
   delNotifiedFIFO_elt(req);
 }
```

### **(&#x33;)** Adding traces to the code

To make sure the data is passed (bach and forth) through the `Thread Pool` a trace has been added and will be removed.

- [ ] Define the trace

```
#include <inttypes.h>
#define DEBUG_PUSCH_THREAD
```

- [ ] Implement tracing

* Trace after initialization

```c
#ifdef DEBUG_PUSCH_THREAD

 for (int aarx=0; aarx<gNB->frame_parms.nb_antennas_rx; aarx++) {
    printf("Start - Estimated delay = %i\t", delay[aarx].est_delay >> 1);
 }

#endif

* Trace at the Thread Pool exit


```c
 #ifdef DEBUG_PUSCH_THREAD

 printf("\n Exit Pool - Starts with: %i\n",gNB->frame_parms.nb_antennas_rx);
 for (int aarx=0; aarx<gNB->frame_parms.nb_antennas_rx; aarx++) {
  printf("Array # = %i\t Estimated delay = %i\t Noise Amp2 = %" PRIu64 "\t", aarx, delay[aarx].est_delay, noises_amp2[aarx]);
 }
 printf("\n Exit Pool - Ends \n");

#endif
```


#### **&#x1F516;** **(&#x62;)** ___Compiling and Running the tests___

---



#### **(&#x33;)** Build the code using the PHY Simulators

- [ ] Build

```
./cmake_targets/build_oai --phy_simulators
```

- [ ] Test

* When testing with a single antenna (-z 1)

```
sudo cmake_targets/ran_build/build/nr_ulsim -C 4 -m 25 -s 24 -z 1 -n 100 -P -q 1 -R 273 -r 273
```

* When testing with a multiple antennas (-z 4)

```
sudo cmake_targets/ran_build/build/nr_ulsim -C 4 -m 25 -s 24 -z 4 -n 100 -P -q 1 -R 273 -r 273
```

* When testing with a multiple antennas (-z 4) and multiple cores (-C 8)

```
sudo cmake_targets/ran_build/build/nr_ulsim -C 8 -m 25 -s 24 -z 8 -n 100 -P -q 1 -R 273 -r 273
```

#### **&#x1F516;** **(&#x63;)** ___Impacted source code___

---


- [ ] Headers [defs_gNB.h](https://gitlab.eurecom.fr/oai/openairinterface5g/-/blob/dmrs_channel_estimation_parallelization/openair1/PHY/defs_gNB.h)

```c
typedef struct {
  PHY_VARS_gNB *gNB;
  int aarx;

...

} puschAntennaProc_t;
```

```c
/// Top-level PHY Data Structure for gNB
typedef struct PHY_VARS_gNB_s {
  /// Module ID indicator for this instance
  module_id_t          Mod_id;

...
  notifiedFIFO_t respPuschAarx;
...
  int nbAarx;
...
} PHY_VARS_gNB;
```

- [ ] Source Code [nr_ul_channel_estimation.c](https://gitlab.eurecom.fr/oai/openairinterface5g/-/blob/dmrs_channel_estimation_parallelization/openair1/PHY/NR_ESTIMATION/nr_ul_channel_estimation.c)

See **&#x1F516;** **(&#x61;)** for coding implementation

#### **&#x1F516;** **(&#x64;)** ___Results___

---

When looking at the `nr_ulsim` test output, there are $\color{salmon} \text{no significant}$ improvements. Further testing needs to be conducted.

&#x1F9EA; Before: 4 Antennas

```powershell
Total PHY proc rx                           1251.82 us (122 trials)
 Statistics std=123.97, median=0.00, q1=0.00, q3=0.00 µs (on 0 trials)
|__ RX PUSCH time                          375.21 us (122 trials)
    |__ ULSCH channel estimation time      152.48 us (122 trials)
...
```

&#x1F9EB; After: 4 Antennas

```powershell
Total PHY proc rx                           1202.96 us (103 trials)
 Statistics std=36.29, median=0.00, q1=0.00, q3=0.00 µs (on 0 trials)
|__ RX PUSCH time                          334.40 us (103 trials)
    |__ ULSCH channel estimation time      126.46 us (103 trials)
...
```

---

- Main output

```powershell
CMDLINE: "cmake_targets/ran_build/build/nr_ulsim" "-C" "4" "-m" "25" "-s" "24" "-z" "4" "-n" "100" "-P" "-q" "1" "-R" "273" "-r" "273" 
[CONFIG] get parameters from cmdline [CONFIG] debug flags: 0x00400000
Initializing random number generator, seed 16645839219334760675
handling optarg C
handling optarg m
handling optarg s
Setting SNR0 to 24.000000
handling optarg z
handling optarg n
handling optarg P
handling optarg q
handling optarg R
handling optarg r
[CONFIG] log_config: 2/3 parameters successfully set 
[CONFIG] log_config: 50/50 parameters successfully set 
[CONFIG] log_config: 50/50 parameters successfully set 
[CONFIG] log_config: 16/16 parameters successfully set 
[CONFIG] log_config: 16/16 parameters successfully set 
log init done
create a thread for core -1
create a thread for core -1
create a thread for core -1
create a thread for core -1
DL frequency 3649140000: band 48, UL frequency 3649140000
[PHY]   Init: N_RB_DL 273, first_carrier_offset 2458, nb_prefix_samples 288,nb_prefix_samples0 352, ofdm_symbol_size 4096
[CONFIG] loader: 2/2 parameters successfully set 
[CONFIG] loader.dfts: 1/2 parameters successfully set 
shlib_path libdfts.so
[LOADER] library libdfts.so successfully loaded
[CONFIG] loader.ldpc: 1/2 parameters successfully set 
shlib_path libldpc.so
[LOADER] library libldpc.so successfully loaded
AWGN: ricean_factor 0.000000
[CONFIG] loader.dfts: 1/2 parameters successfully set 
shlib_path libdfts.so
[LOADER] library libdfts.so has been loaded previously, reloading function pointers
[LOADER] library libdfts.so successfully loaded
num dmrs sym 1
[ULSIM]: length_dmrs: 1, l_prime_mask: 1	number_dmrs_symbols: 1, mapping_type: 1 add_pos: 0 
[ULSIM]: CDM groups: 1, dmrs_config_type: 0, num_rbs: 273, nb_symb_sch: 12
[ULSIM]: MCS: 25, mod order: 8, code_rate: 8850

[ULSIM]: VALUE OF G: 301392, TBS: 262376

*****************************************
SNR 33.200000: n_errors (3/100,0/3,0/0,0/0) (negative CRC), false_positive 0/100, errors_scrambling (23793/30139200,756/904176,0/0,0/0)

SNR 33.200000: Channel BLER (3.000000e-02,0.000000e+00,-nan,-nan Channel BER (7.894370e-04,8.361204e-04,-nan,-nan) Avg round 1.03, Eff Rate 258440.3600 bits/slot, Eff Throughput 98.50, TBS 262376 bits/slot
DMRS-PUSCH delay estimation: min 0, max 0, average 0.000000
*****************************************

gNB RX
Total PHY proc rx                           1202.96 us (103 trials)
 Statistics std=36.29, median=0.00, q1=0.00, q3=0.00 µs (on 0 trials)
|__ RX PUSCH time                          334.40 us (103 trials)
    |__ ULSCH channel estimation time      126.46 us (103 trials)
    |__ RX PUSCH Initialization time        49.05 us (103 trials)
    |__ RX PUSCH Symbol Processing time    158.45 us (103 trials)

UE TX
|__ ULSCH total encoding time              330.40 us (103 trials)
    |__ ULSCH segmentation time             20.56 us (4700 trials)
    |__ ULSCH LDPC encoder time              0.00 us (  1 trials)
    |__ ULSCH rate-matching time             0.73 us (3296 trials)
    |__ ULSCH interleaving time              3.00 us (3296 trials)
|__ RX SRS time                              0.00 us (  0 trials)
    |__ Generate SRS sequence time           0.00 us (  0 trials)
    |__ Get SRS signal time                  0.00 us (  0 trials)
    |__ SRS channel estimation time          0.00 us (  0 trials)
    |__ SRS timing advance estimation time   0.00 us (  0 trials)
    |__ SRS report TLV build time            0.00 us (  0 trials)
        |__ SRS beam report build time       0.00 us (  0 trials)
        |__ SRS IQ matrix build time         0.00 us (  0 trials)
        
gNB RX
Total PHY proc rx                           1202.96 us (103 trials)
 Statistics std=36.29, median=0.00, q1=0.00, q3=0.00 µs (on 0 trials)
|__ RX PUSCH time                          334.40 us (103 trials)
    |__ ULSCH channel estimation time      126.46 us (103 trials)
    |__ RX PUSCH Initialization time        49.05 us (103 trials)
    |__ RX PUSCH Symbol Processing time    158.45 us (103 trials)

UE TX
|__ ULSCH total encoding time              330.40 us (103 trials)
    |__ ULSCH segmentation time             20.56 us (4700 trials)
    |__ ULSCH LDPC encoder time              0.00 us (  1 trials)
    |__ ULSCH rate-matching time             0.73 us (3296 trials)
    |__ ULSCH interleaving time              3.00 us (3296 trials)
|__ RX SRS time                              0.00 us (  0 trials)
    |__ Generate SRS sequence time           0.00 us (  0 trials)
    |__ Get SRS signal time                  0.00 us (  0 trials)
    |__ SRS channel estimation time          0.00 us (  0 trials)
    |__ SRS timing advance estimation time   0.00 us (  0 trials)
    |__ SRS report TLV build time            0.00 us (  0 trials)
        |__ SRS beam report build time       0.00 us (  0 trials)
        |__ SRS IQ matrix build time         0.00 us (  0 trials)

*****************************************
SNR 33.400000: n_errors (0/100,0/0,0/0,0/0) (negative CRC), false_positive 0/100, errors_scrambling (24622/30139200,0/0,0/0,0/0)

SNR 33.400000: Channel BLER (0.000000e+00,-nan,-nan,-nan Channel BER (8.169427e-04,-nan,-nan,-nan) Avg round 1.00, Eff Rate 262376.0000 bits/slot, Eff Throughput 100.00, TBS 262376 bits/slot
DMRS-PUSCH delay estimation: min 0, max 0, average 0.000000
*****************************************

gNB RX
Total PHY proc rx                           1204.53 us (100 trials)
 Statistics std=10.28, median=0.00, q1=0.00, q3=0.00 µs (on 0 trials)
|__ RX PUSCH time                          333.31 us (100 trials)
    |__ ULSCH channel estimation time      125.86 us (100 trials)
    |__ RX PUSCH Initialization time        49.01 us (100 trials)
    |__ RX PUSCH Symbol Processing time    158.01 us (100 trials)

UE TX
|__ ULSCH total encoding time              336.58 us (100 trials)
    |__ ULSCH segmentation time             20.56 us (4800 trials)
    |__ ULSCH LDPC encoder time              0.00 us (  1 trials)
    |__ ULSCH rate-matching time             0.74 us (3200 trials)
    |__ ULSCH interleaving time              2.99 us (3200 trials)
|__ RX SRS time                              0.00 us (  0 trials)
    |__ Generate SRS sequence time           0.00 us (  0 trials)
    |__ Get SRS signal time                  0.00 us (  0 trials)
    |__ SRS channel estimation time          0.00 us (  0 trials)
    |__ SRS timing advance estimation time   0.00 us (  0 trials)
    |__ SRS report TLV build time            0.00 us (  0 trials)
        |__ SRS beam report build time       0.00 us (  0 trials)
        |__ SRS IQ matrix build time         0.00 us (  0 trials)

*************
PUSCH test OK
*************

Num RB:	273
Num symbols:	12
MCS:	25
DMRS config type:	0
DMRS add pos:	0
PUSCH mapping type:	1
DMRS length:	1
DMRS CDM gr w/o data:	1
```

#### **&#x1F516;** **(&#x65;)** ___Debugging___

---

The below lines are traces extracted from the output to make sure flow of data is hapenning between main thread and thread pool.

Note: This is just an excerpt.
- Estimated delay = -20
- Estimated delay = -40
- Noise Amp2 = 129768
- Noise Amp2 = 145728

---

```powershell
delay = 0	Start - Estimated delay = 0	
 Exit Pool - Starts with: 4
Array # = 0	 Estimated delay = 0	 Noise Amp2 = 119384	Array # = 1	 Estimated delay = -40	 Noise Amp2 = 119384	Array # = 2	 Estimated delay = 0	 Noise Amp2 = 119384	Array # = 3	 Estimated delay = 0	 Noise Amp2 = 119384	
 Exit Pool - Ends 
Start - Estimated delay = 0	Start - Estimated delay = -20	Start - Estimated delay = 0	Start - Estimated delay = 0	
 Exit Pool - Starts with: 4
Array # = 0	 Estimated delay = 0	 Noise Amp2 = 129768	Array # = 1	 Estimated delay = -40	 Noise Amp2 = 129768	Array # = 2	 Estimated delay = 0	 Noise Amp2 = 129768	Array # = 3	 Estimated delay = 0	 Noise Amp2 = 129768	
 Exit Pool - Ends 
Start - Estimated delay = 0	Start - Estimated delay = -20	Start - Estimated delay = 0	Start - Estimated delay = 0	
 Exit Pool - Starts with: 4
Array # = 0	 Estimated delay = 0	 Noise Amp2 = 134168	Array # = 1	 Estimated delay = -40	 Noise Amp2 = 134168	Array # = 2	 Estimated delay = 0	 Noise Amp2 = 134168	Array # = 3	 Estimated delay = 0	 Noise Amp2 = 134168	
 Exit Pool - Ends 
Start - Estimated delay = 0	Start - Estimated delay = -20	Start - Estimated delay = 0	Start - Estimated delay = 0	
 Exit Pool - Starts with: 4
Array # = 0	 Estimated delay = 0	 Noise Amp2 = 126716	Array # = 1	 Estimated delay = -40	 Noise Amp2 = 126716	Array # = 2	 Estimated delay = 0	 Noise Amp2 = 126716	Array # = 3	 Estimated delay = 0	 Noise Amp2 = 126716	
 Exit Pool - Ends 
Start - Estimated delay = 0	Start - Estimated delay = -20	Start - Estimated delay = 0	Start - Estimated delay = 0	
 Exit Pool - Starts with: 4
Array # = 0	 Estimated delay = 0	 Noise Amp2 = 145728	Array # = 1	 Estimated delay = -40	 Noise Amp2 = 140152	Array # = 2	 Estimated delay = 0	 Noise Amp2 = 131336	Array # = 3	 Estimated delay = 0	 Noise Amp2 = 131336	
 Exit Pool - Ends 
...
```